<a href="https://colab.research.google.com/github/FKz11/NN_PyTorch/blob/main/lesson_9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# lesson-9

### Библиотеки:

In [ ]:
!pip install -qq transformers

     |████████████████████████████████| 4.7 MB 27.3 MB/s 
     |████████████████████████████████| 6.6 MB 52.4 MB/s 
     |████████████████████████████████| 596 kB 55.7 MB/s 
     |████████████████████████████████| 101 kB 8.4 MB/s 


In [ ]:
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset

from tqdm import tqdm

from transformers import AutoTokenizer, AutoModel, pipeline

### Данные

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip -qq '/content/drive/MyDrive/Colab Notebooks/data.zip'

In [ ]:
train_df = pd.read_csv('/content/train.csv')
test_df = pd.read_csv('/content/val.csv')
train_df.drop('id', axis=1, inplace=True)
test_df.drop('id', axis=1, inplace=True)
train_df.head()

,text,class
0,@alisachachka не уезжаааааааай. :(❤ я тоже не ...,0
1,RT @GalyginVadim: Ребята и девчата!\nВсе в кин...,1
2,RT @ARTEM_KLYUSHIN: Кто ненавидит пробки ретви...,0
3,RT @epupybobv: Хочется котлету по-киевски. Зап...,1
4,@KarineKurganova @Yess__Boss босапопа есбоса н...,1


In [ ]:
train_df.shape

(181467, 2)

In [ ]:
test_df.shape

(22683, 2)

In [ ]:
train_df['class'].value_counts()

1    92063
0    89404
Name: class, dtype: int64

Дисбаланса классов нету

## 1. Возьмите готовую модель из https://huggingface.co/models для классификации сентимента текста.

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device_int = 0 if torch.cuda.is_available() else -1
device

'cuda'

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("Tatyana/rubert-base-cased-sentiment-new")
base_model = AutoModel.from_pretrained("Tatyana/rubert-base-cased-sentiment-new")

sentiment = pipeline("text-classification", model='Tatyana/rubert-base-cased-sentiment-new', return_all_scores=True, device=device_int)
sentiment("Обычный день")

Some weights of the model checkpoint at Tatyana/rubert-base-cased-sentiment-new were not used when initializing BertModel: ['classifier.weight', 'classifier.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/usr/local/lib/python3.7/dist-packages/transformers/pipelines/text_classification.py:90: UserWarning: `return_all_scores` is now deprecated, use `top_k=1` if you want similar functionnality
  "`return_all_scores` is now deprecated, use `top_k=1` if you want similar functionnality", UserWarning


[[{'label': 'NEUTRAL', 'score': 0.49175795912742615},
  {'label': 'POSITIVE', 'score': 0.21006052196025848},
  {'label': 'NEGATIVE', 'score': 0.29818156361579895}]]

In [ ]:
base_model

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(119547, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
         

## 2. Сделайте предсказания на всем df_val. Посчитайте метрику качества.

In [ ]:
tokens = tokenizer(test_df['text'].tolist())['input_ids']
tokens[0]

[101,
 7155,
 168,
 20620,
 253,
 10874,
 207,
 14730,
 4416,
 1330,
 156,
 60653,
 48512,
 106,
 122,
 122,
 122,
 110,
 545,
 1790,
 60556,
 47829,
 156,
 134,
 134,
 270,
 132,
 11682,
 134,
 276,
 29395,
 203,
 137,
 279,
 183,
 213,
 10267,
 102]

In [ ]:
max_len = max([len(i) for i in tokens])
max_len

118

In [ ]:
mean_len = sum([len(i) for i in tokens]) / len(tokens)
mean_len

27.679407485782303

MAX_LENGTH = 118

In [ ]:
MAX_LENGTH = 118

In [ ]:
%%time

test_df['pred'] = sentiment(test_df['text'].tolist(), padding='max_length', max_length=MAX_LENGTH, truncation=True)

CPU times: user 3min 52s, sys: 1.13 s, total: 3min 53s
Wall time: 3min 55s


In [ ]:
test_df['pred'][0]

[{'label': 'NEUTRAL', 'score': 0.005960037000477314},
 {'label': 'POSITIVE', 'score': 0.990689754486084},
 {'label': 'NEGATIVE', 'score': 0.0033502373844385147}]

In [ ]:
test_df['pred'] = test_df['pred'].apply(lambda res: int(res[1]['score'] > res[2]['score']))
test_df['pred'].head()

0    1
1    0
2    0
3    0
4    0
Name: pred, dtype: int64

In [ ]:
test_acc = (test_df['class'] == test_df['pred']).sum() / len(test_df['class'])
test_acc

0.9957677555878852

In [ ]:
errors = test_df[test_df['class'] != test_df['pred']]
errors.head()

,text,class,pred
569,Что в этих ваших дебианах используют для вебка...,1,0
642,"@Echelon_2001 спасибо ^_^ но все-равно, как-то...",1,0
805,@BOMER_BVB я умею смеяться по испански ахахаха...,1,0
992,Хочу пожелать доброго утра и хорошего дня для ...,1,0
1423,^_^с этим просто не сравница^_^ http://t.co/7m...,1,0


In [ ]:
print("Type 1 error (FP): ", len(errors[errors['class'] == 0]))
print("Type 2 error (FN): ", len(errors[errors['class'] == 1]))

Type 1 error (FP):  11
Type 2 error (FN):  85


MAX_LENGTH = 30

In [ ]:
MAX_LENGTH = 30

In [ ]:
%%time

test_df['pred'] = sentiment(test_df['text'].tolist(), padding='max_length', max_length=MAX_LENGTH, truncation=True)

CPU times: user 3min 36s, sys: 1.23 s, total: 3min 38s
Wall time: 3min 40s


In [ ]:
test_df['pred'][0]

[{'label': 'NEUTRAL', 'score': 0.005960250273346901},
 {'label': 'POSITIVE', 'score': 0.9906888008117676},
 {'label': 'NEGATIVE', 'score': 0.0033509007189422846}]

In [ ]:
test_df['pred'] = test_df['pred'].apply(lambda res: int(res[1]['score'] > res[2]['score']))
test_df['pred'].head()

0    1
1    0
2    0
3    0
4    0
Name: pred, dtype: int64

In [ ]:
test_acc = (test_df['class'] == test_df['pred']).sum() / len(test_df['class'])
test_acc

0.9296389366485914

## 3. Дообучите эту модель на df_train. Посчитайте метрику качества на df_val.

### Dataset

In [ ]:
class Dataset(torch.utils.data.Dataset):
    
    def __init__(self, txts, labels, tokenizer, max_length):
        self.labels = torch.LongTensor(labels)
        self.txts = tokenizer(txts, padding='max_length', 
                                max_length=max_length, truncation=True)

    def __getitem__(self, index):
        return self.txts['input_ids'][index], self.txts['attention_mask'][index], self.labels[index]
        
    def __len__(self):
        return len(self.labels)

In [ ]:
train_data = Dataset(train_df['text'].tolist(), train_df['class'], tokenizer, MAX_LENGTH)
test_data = Dataset(test_df['text'].tolist(), test_df['class'], tokenizer, MAX_LENGTH)

### DataLoader

In [ ]:
train_loader = DataLoader(train_data, batch_size=512, shuffle=True, num_workers=2)
test_loader = DataLoader(test_data, batch_size=256, shuffle=True, num_workers=1)

### Создание модели

In [ ]:
class RuBertClassifier(nn.Module):

    def __init__(self, base_model, hidden_dim, dropout):
        super().__init__()
        self.base_model = base_model
        self.dropout = nn.Dropout(dropout)
        self.linear_1 = nn.Linear(768, hidden_dim)
        self.linear_2 = nn.Linear(hidden_dim, 2)

    def forward(self, x, mask):
        
        _, pooled_output = self.base_model(input_ids=x, attention_mask=mask, return_dict=False)
        # _, pooled_output - набор эмбеддинигов слов, эмбеддинг предложения
        dropout_output = self.dropout(pooled_output)
        linear_1_output = self.linear_1(dropout_output)
        tanh_output = torch.tanh(linear_1_output)
        linear_2_output = self.linear_2(tanh_output)
        sigm_output = torch.sigmoid(linear_2_output)
        return sigm_output

In [ ]:
model = RuBertClassifier(base_model, hidden_dim=384, dropout=0.2).to(device)

In [ ]:
for param in model.base_model.parameters():
    param.requires_grad = False

In [ ]:
all_param = sum([param.nelement() for param in model.parameters()])
train_param = all_param - sum([param.nelement() for param in model.base_model.parameters()])
print("Parameters full train:", all_param)
print("Parameters transfer learning:", train_param)

Parameters full train: 178149506
Parameters transfer learning: 296066


### Обучение

In [ ]:
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [ ]:
epochs = 2

for epoch_num in range(epochs):
    train_loss = 0
    test_loss = 0
    train_acc = 0
    test_acc = 0

    model.train()
    for input_id, mask, labels in tqdm(train_loader):
        mask = torch.stack(mask).permute(1,0).to(device)
        input_id = torch.stack(input_id).permute(1,0).to(device)
        labels = labels.to(device)

        output = model(input_id, mask)
                
        loss = criterion(output, labels)
        train_loss += loss.item()
                
        train_acc += (output.argmax(dim=1) == labels).sum().item() / len(labels)

        model.zero_grad()
        loss.backward()
        optimizer.step()
            
    model.eval()
    for input_id, mask, labels in test_loader:
        mask = torch.stack(mask).permute(1,0).to(device)
        input_id = torch.stack(input_id).permute(1,0).to(device)
        labels = labels.to(device)

        output = model(input_id, mask)

        loss = criterion(output, labels)
        test_loss += loss.item()
                    
        test_acc += (output.argmax(dim=1) == labels).sum().item() / len(labels)
            
    print(
        f'Epochs: {epoch_num + 1} | Train Loss: {train_loss / len(train_loader): .3f} \
        | Train Accuracy: {train_acc / len(train_loader): .6f} \
        | Test Loss: {test_loss / len(test_loader): .3f} \
        | Test Accuracy: {test_acc / len(test_loader): .6f}')

100%|██████████| 355/355 [05:01<00:00,  1.18it/s]


Epochs: 1 | Train Loss:  0.381         | Train Accuracy:  0.926272         | Test Loss:  0.378         | Test Accuracy:  0.929959


100%|██████████| 355/355 [05:01<00:00,  1.18it/s]


Epochs: 2 | Train Loss:  0.379         | Train Accuracy:  0.929047         | Test Loss:  0.378         | Test Accuracy:  0.930279


## Вывод:

Я загрузил RuBert модель из библиотеки transformers и использовал её для бинарной задачи прогнозирования эмоциональной окраски текста (sentiment analysis). 

Если брать все слова в предложениях, то есть MAX_LENGTH = 118, результаты впечатляют, 0.9957677555878852 точность предсказания (accuracy). При этом можно заметить, что ошибок первого рода (FP) гораздо меньше ошибок второго рода (FN). 

Если брать MAX_LENGTH = 30, то метрика уже меньше, но всё равно большая, 0.9296389366485914 accuracy. Попробуем эту метрику увеличить с помошью дообучения модели на train, а точнее обучения классификатора для обученной базовой модели. 

В виде классификатора возьмём два линейных слоя, при этом будет 2 выхода с активацией sigmoid, вероятность для каждого класса, хотя можно было обойтись одним выходом, но тогда пришлось бы подбирать порого(trashold), хотя у нас сбалансированные данные и его можно было взять 0.5, это бы всё равно не являлось точным показателем и лучше тогда использовать метрики auc. 

После дообучения видно, что Train Loss падает, Train Accuracy растёт, Test Accuracy растёт, значит наша модель правильно дообучается. Так же видно, что итоговая метрика Test Accuracy с дообучением больше, чем без, значит стратегия дообучения классификатора модели для конкретных данных работает и так стоит делать.